In [1]:
!pip install catboost

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from IPython.display import Image
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import KFold
#from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
#from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family = 'D2coding')
mpl.rc('axes', unicode_minus=False)

sns.set(font="malgun gothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10, 8))

warnings.filterwarnings('ignore')

In [22]:
data = pd.read_csv('C:/k-digital/study/oil/train.csv')
data1 = pd.read_csv('C:/k-digital/study/oil/train.csv')
test = pd.read_csv('C:/k-digital/study/oil/test.csv')
data

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [5]:
data_features = ['YEAR','COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 
                       'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

data = data[data_features]
# test = test[data_features]
# 'COMPONENT_ARBITRARY', 'ANONYMOUS_1'

In [23]:
test_features = ['YEAR','COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 
                       'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']
test = test[test_features]

In [6]:
data.shape

(14095, 18)

In [7]:
data.head(3)

,YEAR,COMPONENT_ARBITRARY,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2011,COMPONENT3,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,2021,COMPONENT2,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,2015,COMPONENT2,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412


# 라벨인코딩

In [8]:
le = LabelEncoder()
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
for col in categorical_features:    
    data[col] = le.fit_transform(data[col])

In [9]:
data['YEAR'].unique()

array([ 4, 14,  8,  3,  1,  2,  7, 11,  6,  9, 10,  5, 12,  0, 15, 13],
      dtype=int64)

In [10]:
data.head(3)

,YEAR,COMPONENT_ARBITRARY,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,4,2,1486,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,14,1,1350,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,8,1,2415,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412


In [11]:
# 스케일링
data_need_scale = data.drop(columns = ['COMPONENT_ARBITRARY', 'YEAR'])

ss = StandardScaler()
data_scaled = ss.fit_transform(data_need_scale)
data_scaled

array([[-0.39376339, -0.3407601 , -0.15021362, ..., -0.1065502 ,
         0.89989172, -0.96600175],
       [-0.42602191, -0.0225763 , -0.15021362, ..., -0.1065502 ,
        -1.31737552,  0.11914661],
       [-0.17340923, -0.3407601 , -0.15021362, ..., -0.1065502 ,
        -0.74088603, -0.33221493],
       ...,
       [-0.3221305 ,  0.29560751, -0.15021362, ..., -0.1065502 ,
        -1.11580577,  1.33970343],
       [-0.15372204,  0.29560751, -0.15021362, ..., -0.1065502 ,
        -1.31132842,  0.11914661],
       [-0.29509027, -0.3407601 , -0.15021362, ..., -0.1065502 ,
        -1.25690459,  0.04391969]])

In [12]:
data_need_scale.columns

Index(['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN'],
      dtype='object')

In [13]:
data_name = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']


data_df = pd.DataFrame(data_scaled, columns = data_name)
data_df

,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,-0.393763,-0.340760,-0.150214,-0.089633,0.339245,0.336858,1.331290,-0.041588,1.186914,-0.384284,1.384414,5.293270,0.622282,-0.10655,0.899892,-0.966002
1,-0.426022,-0.022576,-0.150214,-0.089633,-0.115388,-0.027612,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.259244,-0.102635,-0.10655,-1.317376,0.119147
2,-0.173409,-0.340760,-0.150214,-0.089633,-0.080416,-0.252497,-0.326655,-0.041588,-0.250456,-0.400998,-0.191804,-0.260552,-0.102635,-0.10655,-0.740886,-0.332215
3,1.006399,-0.340760,-0.150214,-0.089633,-0.115388,-0.260252,-0.264764,-0.041588,-0.160621,-0.400998,-0.191804,-0.242884,-0.102635,-0.10655,0.482642,-1.093888
4,0.191634,-0.340760,-0.150214,-0.089633,-0.115388,-0.268007,-0.200996,-0.041588,-0.250456,-0.400998,-0.191804,-0.129674,-0.102635,-0.10655,0.478611,-0.866326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,-0.362928,-0.340760,-0.150214,-0.089633,-0.115388,-0.244743,-0.291021,-0.041588,-0.250456,-0.400998,-0.191804,-0.248773,-0.102635,-0.10655,0.524972,-1.076961
14091,-0.085884,-0.340760,-0.150214,-0.089633,-0.115388,-0.252497,-0.311651,-0.041588,-0.250456,3.342831,-0.191804,-0.265787,-0.102635,-0.10655,0.164162,1.540935
14092,-0.322130,0.295608,-0.150214,-0.089633,0.024499,-0.213724,0.444177,-0.041588,0.378393,-0.233862,0.070899,0.150406,-0.102635,-0.10655,-1.115806,1.339703
14093,-0.153722,0.295608,-0.150214,-0.089633,-0.115388,1.050289,-0.298523,-0.041588,-0.250456,-0.384284,-0.191804,-0.264479,-0.102635,-0.10655,-1.311328,0.119147


In [14]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
data_label = data[categorical_features]
data_new = pd.concat([data_label,data_df], axis=1)

In [15]:
data_new.head(3)

,COMPONENT_ARBITRARY,YEAR,ANONYMOUS_1,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2,4,-0.393763,-0.340760,-0.150214,-0.089633,0.339245,0.336858,1.331290,-0.041588,1.186914,-0.384284,1.384414,5.293270,0.622282,-0.10655,0.899892,-0.966002
1,1,14,-0.426022,-0.022576,-0.150214,-0.089633,-0.115388,-0.027612,-0.330406,-0.041588,-0.250456,-0.400998,-0.191804,-0.259244,-0.102635,-0.10655,-1.317376,0.119147
2,1,8,-0.173409,-0.340760,-0.150214,-0.089633,-0.080416,-0.252497,-0.326655,-0.041588,-0.250456,-0.400998,-0.191804,-0.260552,-0.102635,-0.10655,-0.740886,-0.332215


In [16]:
answer = data1['Y_LABEL'] 

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_new, answer, test_size = 0.2, stratify = answer, random_state = 2022)

# voting

In [18]:
# 부팅 분류기 Voting
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


lg = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors =3)


voting_model = VotingClassifier(estimators = [('lg', lg), ('knn',knn)], voting = 'soft')

voting_model.fit(X_train, y_train)
pred = voting_model.predict(X_test)

In [19]:
# 각 모델 에측과 평가
models = [lg, knn]
for model in models :
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    model_name = model.__class__.__name__
    print('%s 정확도 : %.3f' %(model_name, accuracy_score(pred, y_test)))

LogisticRegression 정확도 : 0.914
KNeighborsClassifier 정확도 : 0.903


In [20]:
from sklearn.metrics import f1_score

In [21]:
print('Macro f1 socre : ', f1_score(y_test, pred, labels=[0,1], average='macro'))

Macro f1 socre :  0.5265936582184763


# TEST

In [ ]:
test